## Data 전처리

### 기업 통일

2016년을 기준으로 2017,2018,2019의 기업을 통일 시킬 것이다.

2016년 이후 신규 창립기업은 모델 학습, 테스트에 사용하지 않을 것이며, 파산기업은 파산한 시점부터 모든 값들을 0으로 처리한다.

In [2]:
import pandas as pd
import numpy as np

In [3]:
standard_data = pd.read_excel('C:/Users/JSLee/Desktop/KYU/2017.01.xlsx', encoding= "utf-8")

In [4]:
standard_data.head()

,Code,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),1일\n등락률\n(%),5일\n등락률\n(%),20일\n등락률\n(%),...,16년 3Q.2,16년 4Q(E).2,17년 1Q(E).2,16년4Q \n매출액,16년4Q \n영업이익,16년4Q \n순이익,16년4Q \n매출액.1,16년4Q \n영업이익.1,16년4Q \n순이익.1,16년4Q \n순이익.2
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,1833000,2578652,-2.135611,-1.504567,2.230898,...,44087.90,64474.315237,66685.180236,530000.0,92000.0,NaN,521120.451352,83715.404235,64474.315237,64474.315237
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,49300,358905,-1.988072,-0.504541,6.250000,...,5967.74,10690.789385,12736.206023,NaN,NaN,NaN,51332.259150,14568.549707,10690.789385,10690.789385
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,150500,331516,1.006711,0.000000,7.885305,...,10614.40,13513.247779,14784.204410,NaN,NaN,NaN,247224.041550,13353.916707,13513.247779,13513.247779
3,A015760,한국전력,유틸리티,전력,코스피,44400,285032,-0.448430,2.896871,-3.687636,...,29133.61,13654.365561,17147.707040,NaN,NaN,NaN,147316.591167,24639.338148,13654.365561,13654.365561
4,A035420,NAVER,IT서비스,인터넷서비스,코스피,794000,261724,0.506329,1.925546,-1.488834,...,1945.48,2117.854524,2221.450690,NaN,NaN,NaN,10916.344137,3030.094202,2117.854524,2117.854524


In [5]:
test_data = pd.read_excel('C:/Users/JSLee/Desktop/KYU/2019.06.xlsx', encoding= "utf-8")

In [ ]:
test_data.head()

In [ ]:
type(test_data['Code'])

In [ ]:
standard_bis = set(standard_data['Code'])

In [ ]:
test_bis = set(test_data['Code'])

In [ ]:
test_bis - standard_bis

In [ ]:
drop_bis = test_bis - standard_bis  #신규 발행 회사 제거
for k in drop_bis:
    id = test_data[test_data['Code']==k].index
    test_data.drop(id,inplace = True)

In [ ]:
test_data.head()

In [ ]:
#제대로 기업이 빠졌는지 확인

if 'A002800' in set(test_data['Code']):
    print('Fail..')
else:
    print('Success')

In [ ]:
test_data

In [ ]:
def select_bis(standard_data,test_data):  #standard_data : 2017.01월 데이터, test_bis : 원하는 일시의 데이터
    standard_bis = set(standard_data['Code'])
    test_bis = set(test_data['Code'])
    drop_bis = test_bis - standard_bis  #신규 발행 회사 제거
    for k in drop_bis:
        id = test_data[test_data['Code']==k].index
        test_data.drop(id,inplace = True)
        
    return test_data

### 피쳐 통일

In [6]:
reverse_st_data = standard_data.T
reverse_te_data = test_data.T
standard_data.T.head()
test_data.T.head()

,0,1,2,3,4,5,6,7,8,9,...,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102
Code,A005930,A000660,A051910,A005380,A068270,A005490,A207940,A051900,A055550,A012330,...,A257730,A311270,A279410,A310870,A310840,A258790,A264290,A310200,A065160,A149940
회사명,삼성전자,SK하이닉스,LG화학,현대차,셀트리온,POSCO,삼성바이오로직스,LG생활건강,신한지주,현대모비스,...,신한제3호스팩,키움제5호스팩,한화에이스스팩4호,한국제8호스팩,엔에이치스팩13호,케이비제11호스팩,한화에이스스팩3호,미래에셋대우스팩2호,에프티이앤이,모다
업종\n(대),반도체 관련장비 및 부품,반도체 관련장비 및 부품,화학,자동차 및 관련부품,제약 및 바이오,철강 및 비철강,제약 및 바이오,가정용품,금융,자동차 및 관련부품,...,금융,금융,금융,금융,금융,금융,금융,금융,섬유 및 의류,통신
업종\n(소),종합 반도체,종합 반도체,종합 화학,완성차,바이오,제철,바이오,화장품,은행,기타 자동차부품,...,스팩,스팩,스팩,스팩,스팩,스팩,스팩,스팩,섬유,통신장비
코스피코스닥,코스피,코스피,코스피,코스피,코스피,코스피,코스피,코스피,코스피,코스피,...,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥,코스닥


In [7]:
reverse_st_data.reset_index(inplace = True)
reverse_te_data.reset_index(inplace = True)

In [8]:
standard_features = set(reverse_st_data['index'])
test_features = set(reverse_te_data['index'])
drop_features = test_features - standard_features  #신규 발행 피쳐 제거
for k in drop_features:
    id = reverse_te_data[reverse_te_data['index']==k].index
    reverse_te_data.drop(id,inplace = True)

In [9]:
drop_features

{'15년->18년\n3년간 YOY',
 '15년->18년\n3년간 YOY.1',
 '15년->18년\n3년간 YOY.2',
 '16년\nYOY',
 '16년\nYOY.1',
 '16년\nYOY.2',
 '16년\n매출액',
 '16년\n순이익',
 '16년\n영업이익',
 '17년\nYOY',
 '17년\nYOY.1',
 '17년\nYOY.2',
 '17년\n매출액',
 '17년\n순이익',
 '17년\n영업이익',
 '17년 2Q',
 '17년 2Q.1',
 '17년 2Q.2',
 '17년 3Q',
 '17년 3Q.1',
 '17년 3Q.2',
 '17년 4Q',
 '17년 4Q.1',
 '17년 4Q.2',
 '18년\nYOY',
 '18년\nYOY.1',
 '18년\nYOY.2',
 '18년\n매출액',
 '18년\n순이익',
 '18년\n영업이익',
 '18년 1Q',
 '18년 1Q.1',
 '18년 1Q.2',
 '18년 2Q',
 '18년 2Q.1',
 '18년 2Q.2',
 '18년 3Q',
 '18년 3Q.1',
 '18년 3Q.2',
 '18년 4Q',
 '18년 4Q.1',
 '18년 4Q.2',
 '18년2Q QOQ',
 '18년2Q QOQ.1',
 '18년2Q QOQ.2',
 '18년2Q YOY',
 '18년2Q YOY.1',
 '18년2Q YOY.2',
 '18년3Q QOQ',
 '18년3Q QOQ.1',
 '18년3Q QOQ.2',
 '18년3Q YOY',
 '18년3Q YOY.1',
 '18년3Q YOY.2',
 '18년4Q QOQ',
 '18년4Q QOQ.1',
 '18년4Q QOQ.2',
 '18년4Q YOY',
 '18년4Q YOY.1',
 '18년4Q YOY.2',
 '19/1Q\n매출액\n컨센대비',
 '19/1Q\n순이익\n컨센대비',
 '19/1Q\n영업이익\n컨센대비',
 '19/2Q\n매출액\n컨센대비',
 '19/2Q\n순이익\n컨센대비',
 '19/2Q\n영업이익\n컨센대비',
 '19년 1Q',
 '19년 1

In [10]:
#제대로 피쳐가 빠졌는지 확인

if '총자산\n(억)' in set(reverse_te_data['index']):
    print('Fail..')
else:
    print('Success')

Success


In [12]:
#다시 원래대로 되돌리기

test_data = reverse_te_data.T

In [13]:
test_data

,0,1,2,3,4,5,6,11,12,23,...,125,132,133,134,135,137,138,210,211,212
index,Code,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),1일\n등락률\n(%),5일\n등락률\n(%),과거\nROE\n(%),...,실적\n발표일,1주일\n어닝스,1달\n어닝스,3개월 \n어닝스,6개월 \n어닝스,12개월 \n어닝스,어닝스\n점수,15년\n매출액,15년\n영업이익,15년\n순이익
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,43900,2620735,-0.2,-2,15.2,...,20190405,-0.9,0.7,-19,-36.8,-32.9,0,2006535,264134,186946
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,63700,463738,1,-4.6,28.7,...,20190425,-8.9,-29.1,-52,-73.1,-72.4,0,187980,53361,43224
2,A051910,LG화학,화학,종합 화학,코스피,348500,246014,1.2,3.4,6.7,...,20190424,3,-5.7,-13.6,-19.8,22.5,40,202066,18236,11530
3,A005380,현대차,자동차 및 관련부품,완성차,코스피,140000,299135,-0.4,-2.1,2.4,...,20190424,0.4,8.3,6.7,17,144.3,80,919587,63579,64173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,A258790,케이비제11호스팩,금융,스팩,코스닥,2245,72,0.2,0.7,0,...,20190513,0,0,0,0,0,0,0,0,0
2099,A264290,한화에이스스팩3호,금융,스팩,코스닥,2375,73,-0.6,6,0,...,20190514,0,0,0,0,0,0,0,0,0
2100,A310200,미래에셋대우스팩2호,금융,스팩,코스닥,2465,75,0,2.9,0,...,20190514,0,0,0,0,0,0,0,0,0
2101,A065160,에프티이앤이,섬유 및 의류,섬유,코스닥,0,0,0,0,-71.7,...,0,0,0,0,1.8,538.5,40,482,-182,-223


In [ ]:
def select_feature(standard_data,test_data):  #standard_data : 2017.01월 데이터, test_bis : 원하는 일시의 데이터
    reverse_st_data = standard_data.T
    reverse_te_data = test_data.T
    standard_data.T.head()
    test_data.T.head()
    standard_features = set(reverse_st_data['index'])
    test_features = set(reverse_te_data['index'])
    drop_features = test_features - standard_features  #신규 발행 피쳐 제거
    for k in drop_features:
        id = reverse_te_data[reverse_te_data['index']==k].index
        reverse_te_data.drop(id,inplace = True)
    test_data = reverse_te_data.T
    
    return test_data